<a href="https://colab.research.google.com/github/aburdenko/gcp-jupyter-notebooks/blob/main/voila/voila_earth_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>
<a target="_blank"  href="https://console.cloud.google.com/vertex-ai/workbench/list/instances"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/a/a0/Google_Cloud_Workbench.png" /> Run in Google Cloud Vertex Workbench</a>

Runs on Colab Notebooks available in the [GCP Marketplace](https://console.cloud.google.com/marketplace/product/colab-marketplace-image-public/colab).

In [42]:
import os
import sys

nb_path = '/content/notebooks'
nb_path_lib='/content/notebooks/lib'
if not os.path.islink(nb_path):

  import os, sys
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  # !mkdir -p nb_path

  os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)

# #os.listdir(nb_path+'/lib')
sys.path.insert(0,nb_path_lib)

if '.' not in sys.path:
  sys.path.insert(0, '.')

# Caching python libraries on Google Drive allows you to skip the next cell after first run...

In [43]:
%%capture

import sys
if 'google.colab' in sys.modules:
  USER_FLAG = ''
else:
  USER_FLAG = '--user'

# try:
#     nb_path_lib = do_imports()
# except ImportError as e:    

# nb_path_lib = do_imports()
import os
libs = os.listdir(nb_path_lib)
if 'voila' not in libs \
  or not 'tornado' in libs \
  or not 'geemap' in libs:
  !pip install --quiet $USER_FLAG --target=$nb_path_lib geemap 
  !pip install --quiet $USER_FLAG --target=$nb_path_lib tornado
  !pip install --quiet $USER_FLAG --target=$nb_path_lib voila 
  
  print("🔁 Restarting kernel...")
  get_ipython().kernel.do_shutdown(True)


## Deploy Earth Engine Apps using Voila and LocalTunnel...


In [44]:
import os, sys
nb_path_lib='/content/notebooks/lib'
sys.path.insert(0,nb_path_lib)


if '.' not in sys.path:
  sys.path.insert(0, '.')

import os
import ee
import geemap
import ipywidgets as widgets
import voila 

In [45]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [46]:
style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:',
    value='Landsat Timelapse',
    width=200,
    style=style
)

bands = widgets.Dropdown(
    description='Select RGB Combo:',
    options=['Red/Green/Blue', 'NIR/Red/Green',  'SWIR2/SWIR1/NIR', 'NIR/SWIR1/Red','SWIR2/NIR/Red', 
             'SWIR2/SWIR1/Red', 'SWIR1/NIR/Blue', 'NIR/SWIR1/Blue', 'SWIR2/NIR/Green', 'SWIR1/NIR/Red'],
    value='NIR/Red/Green',
    style=style
)

hbox1 = widgets.HBox([title, bands])
hbox1

In [47]:
speed = widgets.IntSlider(
    description='  Frames per second:',
    tooltip='Frames per second:',
    value=10,
    min=1, 
    max = 30,
    style=style
)

cloud = widgets.Checkbox(
    value=True,
    description='Apply fmask (remove clouds, shadows, snow)',
    style=style
)

hbox2 = widgets.HBox([speed, cloud])
hbox2

In [48]:
start_year = widgets.IntSlider(description='Start Year:', value=1984, min=1984, max=2020, style=style)
end_year = widgets.IntSlider(description='End Year:', value=2020, min=1984, max=2020, style=style)
start_month = widgets.IntSlider(description='Start Month:', value=5, min=1, max=12, style=style)
end_month = widgets.IntSlider(description='End Month:', value=10, min=1, max=12, style=style)
hbox3 = widgets.HBox([start_year, end_year, start_month, end_month])
hbox3

In [49]:
font_size = widgets.IntSlider(description='Font size:', value=30, min=10, max=50, style=style)

font_color = widgets.ColorPicker(
    concise=False,
    description='Font color:',
    value='white',
    style=style
)

progress_bar_color = widgets.ColorPicker(
    concise=False,
    description='Progress bar color:',
    value='blue',
    style=style
)

hbox4 = widgets.HBox([font_size, font_color, progress_bar_color])
hbox4

In [50]:
create_gif = widgets.Button(
    description='Create timelapse',
    button_style='primary',
    tooltip='Click to create timelapse',
    style=style
)

download_gif = widgets.Button(
    description='Download GIF',
    button_style='primary',
    tooltip='Click to download timelapse',
    disabled=False,
    style=style
)

output = widgets.Output()

hbox5 = widgets.HBox([create_gif])
hbox5

In [51]:
def submit_clicked(b):
    
    with output:
        output.clear_output()
        if start_year.value > end_year.value:
            print('The end year must be great than the start year.')
            return
        if start_month.value > end_month.value:
            print('The end month must be great than the start month.')
            return        
        if start_year.value == end_year.value:
            add_progress_bar = False
        else:
            add_progress_bar = True
            
        start_date = str(start_month.value).zfill(2) + '-01'
        end_date = str(end_month.value).zfill(2) + '-30'
        
        print('Computing...')
        
        Map.add_landsat_ts_gif(roi=Map.user_roi, label=title.value, start_year=start_year.value, 
                               end_year=end_year.value, start_date=start_date, end_date=end_date, 
                               bands=bands.value.split('/'), font_color=font_color.value, 
                               frames_per_second=speed.value, font_size=font_size.value, 
                               add_progress_bar= add_progress_bar, progress_bar_color=progress_bar_color.value, 
                               download=True, apply_fmask=cloud.value) 
            
create_gif.on_click(submit_clicked)

In [52]:
output

Output()

In [53]:
%%capture
%%writefile run_voila.sh
#!/bin/bash
LOW_BOUND=49152
RANGE=16384
while true; do
    PORT=$[$LOW_BOUND + ($RANDOM % $RANGE)]
    (echo "" >/dev/tcp/127.0.0.1/${CANDIDATE}) >/dev/null 2>&1
    if [ $? -ne 0 ]; then
        echo $PORT
        break
    fi
done
fuser -k -TERM -n tcp $PORT
voila --port $PORT --no-browser --debug --strip_sources=True /content/notebooks/voila/voila_earth_demo.ipynb & npx localtunnel --port $PORT


In [54]:
%%capture
! chmod +x /content/run_voila.sh > /dev/null 2>&1

In [55]:
%%capture
! nohup /content/run_voila.sh  &

In [56]:
! sleep 6
! tail -1 nohup.out

your url is: https://selfish-gecko-60.loca.lt


Want to share you dashboards and data models enterprise wide? Step up to [Looker](https://console.cloud.google.com/marketplace/product/looker-public/looker-data-platform-saas?project=aburdenko-project) for Enterprise wide data sharing!